# Data Wrangling

## Gather

In [251]:
import pandas as pd
import numpy as np
import requests
import tweepy
import json
import os
from timeit import default_timer as timer

In [278]:
#read csv
twitter_archive = pd.read_csv('twitter-archive-enhanced.txt')
twitter_archive_clean = twitter_archive.copy()

In [279]:
twitter_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [287]:
#get image predictions file from Udacity server and save to current directory
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
response.content
file_type = url.split('/')[-1]

with open(file_type, "wb") as file:
    file.write(response.content)
    
image_predictions = pd.read_csv('image-predictions.tsv', sep='\t', header=0)
image_predictions_clean = image_predictions.copy()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [255]:
#use Twitter API to get retweet and favorite counts for all tweets from WeRateDogs
consumer_key = 'gVxHHXwZyADEN9DusjKXtW4Ba'
consumer_secret = 'LLhR0ONaW0hV5zhdIIzdy3nmCzk6z9gkKqXkDLXikIEiCGa5lv'
access_token = '209898784-KnLgDLy3AiwHPnDFt5hoBUC0Al61R1pMDYgCVbqS'
access_secret = '4AIymGO6kiCK6tR4dqqm5JP4yMFWtZsAyZbsn31EoFa9U'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
# tweet_ids = twitter_archive_clean.tweet_id.values
# len(tweet_ids)

# # Query Twitter's API for JSON data for each tweet ID in the Twitter archive
# count = 0
# fails_dict = {}
# start = timer()
# # Save each tweet's returned JSON as a new line in a .txt file
# with open('tweet_json_copy.txt', 'w') as outfile:
#     # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
#     for tweet_id in tweet_ids:
#         count += 1
#         print(str(count) + ": " + str(tweet_id))
#         try:
#             tweet = api.get_status(tweet_id, tweet_mode='extended')
#             print("Success")
#             json.dump(tweet._json, outfile)
#             outfile.write('\n')
#         except tweepy.TweepError as e:
#             print("Fail")
#             fails_dict[tweet_id] = e
#             pass
# end = timer()
# print(end - start)
# print(fails_dict)


In [263]:
d = []

with open('tweet_json_copy.txt', 'r') as f:
    for line in f:
        tweets = json.loads(line)
        d.append(
            {
                'tweet_id': tweets['id'],
                'retweet_count': tweets['retweet_count'],
                'favorite_count': tweets['favorite_count']
            }
        )

In [267]:
len(d)

2354

In [290]:
json_info = pd.DataFrame(d, columns = ['tweet_id', 'retweet_count', 'favorite_count'])
json_info_clean = tweet_api_info.copy()

In [292]:
json_info_clean.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,8853,39467
1,892177421306343426,6514,33819
2,891815181378084864,4328,25461
3,891689557279858688,8964,42908
4,891327558926688256,9774,41048


## Assess

In [343]:
#review for quality and tidiness issues
twitter_archive_clean.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
305,836260088725786625,nan,nan,2017-02-27 17:01:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Lucy. She spent all morning overseeing...,nan,nan,NaN,https://twitter.com/dog_rates/status/836260088...,13,10,Lucy,None,None,None,None
1749,679047485189439488,nan,nan,2015-12-21 21:15:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This dog doesn't know how to stairs. Quite tra...,nan,nan,NaN,https://twitter.com/dog_rates/status/679047485...,9,10,None,None,None,None,None
6,890971913173991426,nan,nan,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,nan,nan,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
665,790698755171364864,nan,nan,2016-10-24 23:37:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Mosby. He appears to be rather h*ckin ...,nan,nan,NaN,https://twitter.com/dog_rates/status/790698755...,12,10,Mosby,None,None,None,None
127,867900495410671616,nan,nan,2017-05-26 00:29:37 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Unbelievable. We only rate dogs. Please don't ...,nan,nan,NaN,https://twitter.com/dog_rates/status/867900495...,12,10,None,None,None,None,None


In [344]:
#see if any missing data in columns we'll need later
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [329]:
#format so actual values show instead of exponential values
pd.set_option('display.float_format', lambda x: '%.0f' % x)

#check if values make sense
twitter_archive_clean.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2356,78,78,181,181,2356,2356
mean,742771590321719808,745507917855750656,20141706360873208,772039961038007040,12416983653017580,13,10
std,68567047444761032,75824920044192880,125279666255236320,62369278105055600,95992535331517536,46,7
min,666020888022790144,665814696700723200,11856342,666104133288665088,783214,0,0
25%,678398938214475776,675741911993464832,308637449,718631497683582976,4196983835,10,10
50%,719627934716235776,703870840226598912,4196983835,780465709297995776,4196983835,11,10
75%,799337304954252288,825780371286566912,4196983835,820314633777061888,4196983835,12,10
max,892420643555336192,886266357075128320,840547864354918400,887473957103951872,787461778435289088,1776,170


In [338]:
#check for duplicate rows
sum(twitter_archive_clean.duplicated())

0

In [345]:
#review for quality and tidiness issues
image_predictions.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1226,745057283344719872,https://pbs.twimg.com/media/Clb5pLJWMAE-QS1.jpg,2,Shetland_sheepdog,1,True,collie,0,True,Border_collie,0,True
1089,719332531645071360,https://pbs.twimg.com/media/CfuVGl3WEAEKb16.jpg,1,Dandie_Dinmont,0,True,miniature_poodle,0,True,Norfolk_terrier,0,True
1952,863553081350529029,https://pbs.twimg.com/ext_tw_video_thumb/86355...,1,Eskimo_dog,0,True,malamute,0,True,Siberian_husky,0,True
1081,718246886998687744,https://pbs.twimg.com/media/Cfe5tLWXEAIaoFO.jpg,1,Chihuahua,0,True,carton,0,False,Siberian_husky,0,True
1044,712717840512598017,https://pbs.twimg.com/media/CeQVF1eVIAAJaTv.jpg,1,Great_Pyrenees,1,True,kuvasz,0,True,Irish_wolfhound,0,True


In [306]:
#see if any missing data in columns we'll need later
image_predictions_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [331]:
#format so actual values show instead of exponential values
pd.set_option('display.float_format', lambda x: '%.2f' % x)

#check if values make sense
image_predictions_clean.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2075.00,2075.00,2075.00,2075.00,2075.00
mean,738451357852539008.00,1.20,0.59,0.13,0.06
std,67852033330235656.00,0.56,0.27,0.10,0.05
min,666020888022790144.00,1.00,0.04,0.00,0.00
25%,676483507139540992.00,1.00,0.36,0.05,0.02
50%,711998809858043904.00,1.00,0.59,0.12,0.05
75%,793203448525178880.00,1.00,0.84,0.20,0.09
max,892420643555336192.00,4.00,1.00,0.49,0.27


In [339]:
#check for duplicate values
sum(image_predictions_clean.duplicated())

0

In [293]:
#review for quality and tidiness issues
json_info_clean.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,8853,39467
1,892177421306343426,6514,33819
2,891815181378084864,4328,25461
3,891689557279858688,8964,42908
4,891327558926688256,9774,41048


In [310]:
#see if any missing data in columns we'll need later
json_info_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
tweet_id          2354 non-null int64
retweet_count     2354 non-null int64
favorite_count    2354 non-null int64
dtypes: int64(3)
memory usage: 55.3 KB


In [313]:
json_info_clean.sample(5)

,tweet_id,retweet_count,favorite_count
682,788412144018661376,5990,16060
1362,702899151802126337,509,1772
2151,669661792646373376,482,860
353,831262627380748289,2350,13066
744,780092040432480260,2655,0


In [333]:
#format so actual values show instead of exponential values
pd.set_option('display.float_format', lambda x: '%.0f' % x)

#check if values make sense
json_info_clean.describe()

,tweet_id,retweet_count,favorite_count
count,2354,2354,2354
mean,742697816435063552,3165,8081
std,68528120712910648,5285,11815
min,666020888022790144,0,0
25%,678397510244808704,624,1415
50%,719459571111233536,1474,3604
75%,799305849241918464,3652,10122
max,892420643555336192,79515,132810


In [340]:
#check for duplicate values
sum(json_info_clean.duplicated())

0

### Quality

##### Twitter Archive Table
- timestamp has +0000 at the end
- missing 59 expanded urls
- columns should be integers(in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id
- timestamp column should be datetime
- name column has missing values or just a single letter
- some rating numerators are lower than rating denominators
- name column has missing names even though they're in the text column
- no dog stage listed for most dogs 
- all denominators should be 10
- max numerator is 1776

##### Images Table
- p1, p2, p3 names sometimes capitalized, sometimes not 

### Tidiness

##### Twitter Archive Table
- doggo, floofer, pupper, puppo should be in one column, as they're values not variables

##### JSON Table
- retweet_count and favorite_count should be part of twitter_archive table

## Clean

#### Define

#### Code

#### Test

#used to format data
https://stackoverflow.com/questions/55394854/how-to-change-the-format-of-describe-output